In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

from src.inference import fetch_next_hour_predictions

In [14]:
predictions = fetch_next_hour_predictions()

2025-05-04 20:43:00,139 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-04 20:43:00,147 INFO: Initializing external client
2025-05-04 20:43:00,148 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-04 20:43:00,960 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212635
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.68s) 
Current EST time: 2025-05-04 20:43:00.139128
Next hour: 2025-05-04 21:00:00
Found 3 records


In [15]:
import pandas as pd

In [10]:
import numpy as np

In [16]:
station_dict = {
    np.float32(5905.140137): {"name": "Broadway & E 14 St",
           "longitude": -73.99074142,
           "latitude": 40.73454567
           },
    np.float32(6140.049805): {"name": "W 21 St & 6 Ave",
           "longitude": -73.99415556,
           "latitude": 40.74173969},
    np.float32(6822.089844): {"name": "1 Ave & E 68 St",
           "longitude": -73.958115339,
           "latitude": 40.765112281}
}

station_df = pd.DataFrame.from_dict(station_dict, orient='index').reset_index()
station_df.rename(columns={'index': 'start_station_id'}, inplace=True)
merged = pd.merge(predictions, station_df, on='start_station_id', how='left')
merged

,start_station_id,predicted_demand,start_hour,name,longitude,latitude
0,6822.089844,13.0,2025-05-04 21:00:00,1 Ave & E 68 St,-73.958115,40.765112
1,6140.049805,12.0,2025-05-04 21:00:00,W 21 St & 6 Ave,-73.994156,40.741740
2,5905.140137,5.0,2025-05-04 21:00:00,Broadway & E 14 St,-73.990741,40.734546


In [12]:
station_df

,start_station_id,name,longitude,latitude
0,5905.140137,Broadway & E 14 St,-73.990741,40.734546
1,6140.049805,W 21 St & 6 Ave,-73.994156,40.741740
2,6822.089844,1 Ave & E 68 St,-73.958115,40.765112


In [13]:
station_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   start_station_id  3 non-null      float32
 1   name              3 non-null      object 
 2   longitude         3 non-null      float64
 3   latitude          3 non-null      float64
dtypes: float32(1), float64(2), object(1)
memory usage: 216.0+ bytes


In [9]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   start_station_id  3 non-null      float32       
 1   predicted_demand  3 non-null      float64       
 2   start_hour        3 non-null      datetime64[us]
 3   name_x            0 non-null      object        
 4   longitude_x       0 non-null      float64       
 5   latitude_x        0 non-null      float64       
 6   name_y            0 non-null      object        
 7   longitude_y       0 non-null      float64       
 8   latitude_y        0 non-null      float64       
dtypes: datetime64[us](1), float32(1), float64(5), object(2)
memory usage: 336.0+ bytes
